In [27]:
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torchvision import transforms
from PIL import Image
import os
from progressbar import ProgressBar
from PIL import ImageFile
from torchvision.models import vgg16
import torch
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
from multiprocessing import Pool
from string import punctuation
import pickle
import shutil
from PIL import UnidentifiedImageError
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch.cuda
import json
from tqdm import tqdm
import numpy as np


In [12]:
import torch
from torch import optim, nn
from torchvision import models, transforms
model = models.vgg16(pretrained=True)

In [42]:
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
		# Extract VGG-16 Feature Layers
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
		# Extract VGG-16 Average Pooling Layer
    self.pooling = model.avgpool
		# Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
		# Extract the first part of fully-connected layer from VGG16
    self.fc = model.classifier[0]
  
  def forward(self, x):
		# It will take the input 'x' until it returns the feature vector called 'out'
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    #out = self.fc(out) 
    return out 

# Initialize the model
model = models.vgg16(pretrained=True)
new_model = FeatureExtractor(model)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)
new_model.eval()

FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=

In [43]:
transform = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

In [61]:
image = Image.open(open(r'C:\Users\73183\Documents\ProjectX\DepressionMultiModal\baseline_model\new_ds\negative\22032403\1458489236924010498.jpg', 'rb'))
image = image.convert('RGB')  # 将所有图片转换成3通道
x, y = image.size
if x > y:
    x_ = 224 * x // y
    y_ = 224
else:
    x_ = 224
    y_ = 224 * y // x
image = image.resize((x_, y_), Image.ANTIALIAS)
image = transform(image).unsqueeze(0).to(device)
print(image.size())
#image = image[0]


torch.Size([1, 3, 224, 224])


In [59]:
with torch.no_grad():
		# Extract the feature from the image
    feature = new_model(image)
	# Convert to NumPy Array, Reshape it, and save it to features variable

feature.cpu().detach().numpy().reshape(-1)


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [40]:
print(feature)

tensor([[0., 0., 0.,  ..., 0., 0., 0.]])


In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
nltk.download('averaged_perceptron_tagger')
stemmer = PorterStemmer()

def cut_sentence(line):

    words = word_tokenize(line)
    toRet = [] 
    for word in words:
        toRet.append(word)
        if word in ["depression", "depressed"]:
            break
    return toRet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\73183\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
import json
counter = 0
current = os.getcwd()

for user in neg_users:
    text_dir = os.path.join(current,'new_ds', 'negative', user, 'timeline.txt')
    user_counter = 0
    with open(text_dir) as f:

        tweets = f.readlines()
        
        for tweet in tweets:

            j_tweet = json.loads(tweet)
            text = j_tweet['full_text']

            if "depress" in text:
                list = cut_sentence(text)
                sentence = " ".join(list)
                user_counter+= 1
    if user_counter != 0:
        print(user_counter)


1
1
1
1
1
1
1
1
1
2
1
1
2
2
2
1
1
1
2
1
1
1
1
1
2
1
1
1
2
2
1
1
6
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
2
1
1
1
2
1
1
2
1
2
1
1
2
2
1
1
1
2
1
1
20
3
1
1
2
1
1
1


In [24]:
counter

28

In [62]:
with open('final.txt', 'w') as f:
    # print(f.name, image_path[:-4]+'.pkl')
    f.write(str(feature.tolist()))